In [1]:
import omnipath
from omnipath._core.utils._homologene import download_homologene, show_homologene

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import liana as li

In [3]:
import pandas as pd
import numpy as np
from itertools import product

Simplify fun, to be done by a single-column at a time

In [4]:
import pandas as pd

In [5]:
# define a function to replace list elements with dictionary values
def _replace_subunits(lst, my_dict, one_to_many=False):
    result = []
    for x in lst:
        if x in my_dict:
            value = my_dict[x]
            
            if not isinstance(value, list):
                value = [value]
            
            # TODO: one_to_many as integer
            if (not one_to_many) & (len(my_dict[x]) > 1):
                result.append(np.nan)
            else:
                result.append(value)
        else:
            result.append(np.nan)
    return result

In [6]:
def generate_orthologs(data, column, target):
    df = data[[column]].drop_duplicates().set_index(column)
    
    df['subunits'] = df.index.str.split('_')
    df['subunits'] = df['subunits'].apply(_replace_subunits, args=(map_dict, False,))
    df = df['subunits'].explode().reset_index()
    
    grouped = df.groupby(column).filter(lambda x: x['subunits'].notna().all()).groupby(column)
    
    # Generate all possible subunit combinations within each group
    complexes = []
    for name, group in grouped:
        if group[target].isnull().all():
            continue
        subunit_lists = [list(x) for x in group[target]]
        complex_combinations = list(product(*subunit_lists))
        for complex in complex_combinations:
            complexes.append((name, '_'.join(complex)))
    
    # Create output DataFrame
    result = pd.DataFrame(complexes, columns=['source', 'target']).set_index('source')
    
    return result

In [7]:
resource = li.resource.select_resource()
# resource = resource[resource['receptor'].str.contains('_')]

In [8]:
data = li.resource.select_resource()
# resource = resource[resource['receptor'].str.contains('_')]

In [9]:
column = 'receptor'

In [43]:
import omnipath as op

In [44]:
data = op.requests.Annotations.get(resources='SignaLink_pathway', genesymbols=True)

In [45]:
CPLEX_PREFIX = 'COMPLEX:'

In [46]:
data['genesymbol'] = data['genesymbol'].replace(CPLEX_PREFIX, '', regex=True)

In [47]:
column = 'genesymbol'

In [48]:
# data = data[data[column].str.contains('ABI1_')]
data

,uniprot,genesymbol,entity_type,source,label,value,record_id
0,P20963,CD247,protein,SignaLink_pathway,pathway,T-cell receptor,0
1,P43403,ZAP70,protein,SignaLink_pathway,pathway,Receptor tyrosine kinase,1
2,P43403,ZAP70,protein,SignaLink_pathway,pathway,T-cell receptor,2
3,Q9NYJ8,TAB2,protein,SignaLink_pathway,pathway,Toll-like receptor,3
4,Q9NYJ8,TAB2,protein,SignaLink_pathway,pathway,Innate immune pathways,4
...,...,...,...,...,...,...,...
2482,COMPLEX:P00533_Q15303,EGFR_ERBB4,complex,SignaLink_pathway,pathway,JAK/STAT,2482
2483,COMPLEX:P00533_Q15303,EGFR_ERBB4,complex,SignaLink_pathway,pathway,Receptor tyrosine kinase,2483
2484,COMPLEX:P26992_P40189_Q6UWB1,CNTFR_IL27RA_IL6ST,complex,SignaLink_pathway,pathway,JAK/STAT,2484
2485,COMPLEX:Q06330_Q9Y618,NCOR2_RBPJ,complex,SignaLink_pathway,pathway,Notch,2485


In [49]:
map_df = download_homologene('9606', '10090').set_index('source')
map_dict = map_df.groupby(level=0)["target"].apply(list).to_dict()
map_data = generate_orthologs(data, column, 'subunits').sort_values('source')

In [50]:
map_data.head()

,target
source,
AATK,Aatk
ABI1,Abi1
ABL1,Abl1
ABL1_DOK1_NCK1_NCK2,Abl1_Dok1_Nck1_Nck2
ABL2,Abl2


In [51]:
data = data.set_index(column).merge(map_data, left_index=True, right_index=True, how='left').reset_index(names=column)


In [52]:
replace = False
keep_untranslated = True

In [53]:
if replace:
    data[column] = data['target']
    data = data.dropna(subset=[column]).drop('target', axis=1)
elif keep_untranslated:
    data['target'] = data.apply(lambda x: x['target'] if not pd.isnull(x['target']) else x[column], axis=1)

In [54]:
data

,genesymbol,uniprot,entity_type,source,label,value,record_id,target
0,AATK,Q6ZMQ8,protein,SignaLink_pathway,pathway,Receptor tyrosine kinase,1010,Aatk
1,ABI1,Q8IZP0,protein,SignaLink_pathway,pathway,Receptor tyrosine kinase,1026,Abi1
2,ABI1_WASL,COMPLEX:O00401_Q8IZP0,complex,SignaLink_pathway,pathway,Receptor tyrosine kinase,1237,ABI1_WASL
3,ABL1,P00519,protein,SignaLink_pathway,pathway,Receptor tyrosine kinase,96,Abl1
4,ABL1,COMPLEX:P00519,complex,SignaLink_pathway,pathway,Receptor tyrosine kinase,1643,Abl1
...,...,...,...,...,...,...,...,...
2482,ZAP70,P43403,protein,SignaLink_pathway,pathway,Receptor tyrosine kinase,1,Zap70
2483,ZAP70,P43403,protein,SignaLink_pathway,pathway,T-cell receptor,2,Zap70
2484,ZFP36,P26651,protein,SignaLink_pathway,pathway,Receptor tyrosine kinase,721,Zfp36
2485,ZFYVE9,O95405,protein,SignaLink_pathway,pathway,TGF,398,Zfyve9


In [ ]:
def orthology_translate_column(data, column, id_type, target_organism, keep_untranslated=False, source_organism=9606, one_to_many=False):
    # get orthologs
    source_organism, target_organism = str(source_organism), str(target_organism)
    map_df = download_homologene(source_organism, target_organism, id_type).set_index('source')
    map_dict = map_df.groupby(level=0)["target"].apply(list).to_dict()
    map_data = generate_orthologs(data, column, 'subunits').sort_values('source')
    
    # join orthologs
    data = (data.set_index(column).
            merge(map_data, left_index=True, right_index=True, how='left').
            reset_index(names=column)
            )

    
    
    return data

Homology Conversion

In [ ]:
def _explode_complexes(resource: pd.DataFrame,
                       target: str ='source',
                       source: str ='target') -> pd.DataFrame:
    resource['key'] = resource[SOURCE] + '|' + resource[TARGET]
    resource = (resource.set_index('interaction')
                .apply(lambda x: x.str.split('_'))
                .explode([TARGET])
                .explode(SOURCE)
                .reset_index()
                )
    resource[[f'{SOURCE}_complex', f'{TARGET}_complex']] = resource[
        'interaction'].str.split('|', expand=True)

    return resource

In [ ]:
def _rebuild_complexes(df, group_cols, target_col):
    df = resource.copy()
    df_grouped = resource.groupby(group_cols)['target'].agg(lambda x: '_'.join(map(str, x))).reset_index()
    df = df.drop('target', axis=1).merge(df_grouped, on=list(np.setdiff1d(group_cols, target_col)), how='inner')
    df = df.drop(f'{target_col}_complex', axis=1).rename({'target':f'{target_col}_complex'}, axis=1)
    
    return df

In [ ]:
map_df = download_homologene('9606', '10090')

In [ ]:
map_df

In [ ]:
resource = li.resource.select_resource()

In [ ]:
resource = li.resource.explode_complexes(resource)
# map_df = dict(zip(map_df['source'], map_df['target']))

In [ ]:
resource

In [ ]:
resource = resource.merge(map_df, left_on='receptor', right_on='source', how='inner').drop(['source'], axis=1)
resource = _rebuild_complexes(resource, ['interaction', 'ligand'], 'receptor')

In [ ]:
resource

In [ ]:
resource = resource.merge(map_df, left_on='ligand', right_on='source', how='inner').drop(['source'], axis=1)
resource = _rebuild_complexes(resource, ['interaction', 'receptor'], 'ligand')

In [ ]:
resource

In [ ]:
# remove duplicates (1:many mappings)
resource = resource.drop_duplicates(['interaction']).drop(['interaction'], axis=1)

In [ ]:
resource

Only keep full complexes

In [ ]:
map_df = download_homologene('9606', '10090')

In [ ]:
resource = li.resource.select_resource()
resource = li.resource.explode_complexes(resource)
resource = resource.merge(map_df, left_on='receptor', right_on='source', how='inner').drop(['source'], axis=1)

In [ ]:
group_cols = ['interaction', 'ligand']
target = 'receptor'

In [ ]:
resource

In [ ]:
df = resource.copy()

df_grouped = resource.groupby(group_cols)['target'].agg(lambda x: '_'.join(map(str, x))).reset_index()

In [ ]:
df_grouped

In [ ]:
# cplex_table['subunits2']